In [56]:
import pandas as pd
import numpy as np

In [57]:
df=pd.read_csv('FINAL.csv')

In [58]:
df['isOne'] = df['runs_off_bat'].apply(lambda x: 1 if x == 1 else 0)
df['isTwo'] = df['runs_off_bat'].apply(lambda x: 1 if x == 2 else 0)
df['isThree'] = df['runs_off_bat'].apply(lambda x: 1 if x == 3 else 0)
df['isFour'] = df['runs_off_bat'].apply(lambda x: 1 if x == 4 else 0)
df['isSix'] = df['runs_off_bat'].apply(lambda x: 1 if x == 6 else 0)
df['Boundary'] = df['runs_off_bat'].apply(lambda x: 1 if x == 6 or x==4 else 0)
    

In [59]:
df.columns

Index(['match_id', 'innings', 'Unnamed: 2', 'season', 'start_date', 'venue',
       'ball', 'batting_team', 'bowling_team', 'striker', 'non_striker',
       'bowler', 'runs_off_bat', 'extras', 'wides', 'noballs', 'byes',
       'legbyes', 'penalty', 'wicket_type', 'player_dismissed',
       'other_wicket_type', 'other_player_dismissed', 'bowler_type',
       'Total_runs', 'Extraballs', 'over_type', 'number_of_wicket',
       'number_of_dotballs', 'Batting_type', 'isOne', 'isTwo', 'isThree',
       'isFour', 'isSix', 'Boundary'],
      dtype='object')

In [60]:
df['wicket_type']=df['wicket_type'].fillna(0)

In [61]:
def is_bowlers_wicket(player_dismissed, wicket_type):
    if type(player_dismissed) == str:
        if wicket_type not in [ 'run out', 'retired hurt', 'obstructing the field',0]:
            return 1
        else:
            return 0
    else:
        return 0

In [62]:
df['wicket'] = df.apply(lambda x: is_bowlers_wicket(x['player_dismissed'], x['wicket_type']),axis=1)

In [63]:
df['wicket'].unique()

array([0, 1], dtype=int64)

In [64]:
data=df.groupby(['bowler','over_type']).agg({'Total_runs':'sum','number_of_dotballs':'sum','match_id':'nunique','wicket':'sum','Boundary':'sum'}).reset_index().rename({'Total_runs':'Runs_scored','match_id':'Match','wicket':'dismissed'},axis=1)
balls_faced=df.groupby(['bowler','over_type']).agg({'match_id':'count'}).reset_index().rename({'match_id':'Balls_bowled'},axis=1)
final_data=pd.merge(data,balls_faced,on=['bowler','over_type'])

In [65]:
final_data

,bowler,over_type,Runs_scored,number_of_dotballs,Match,dismissed,Boundary,Balls_bowled
0,A Ashish Reddy,Death overs,123,18.0,11,8,13,75
1,A Ashish Reddy,Middle overs,257,59.0,19,10,29,189
2,A Ashish Reddy,PowerPlay,20,2.0,1,0,4,6
3,A Chandila,Death overs,6,1.0,1,0,0,6
4,A Chandila,Middle overs,100,32.0,8,1,10,84
...,...,...,...,...,...,...,...,...
1104,Yuvraj Singh,Death overs,97,19.0,10,6,10,67
1105,Yuvraj Singh,Middle overs,994,239.0,73,30,90,815
1106,Z Khan,Death overs,810,136.0,73,35,90,547
1107,Z Khan,Middle overs,558,100.0,52,15,68,383


In [66]:
final_data['Average']=final_data['Runs_scored']/final_data['dismissed']
final_data['Boundary_per_wicket']=final_data['Boundary']/final_data['dismissed']
final_data['Striker_rate']=(final_data['Balls_bowled']/final_data['dismissed'])
final_data['Balls_per_boundary']=final_data['Balls_bowled']/final_data['Boundary']
final_data.drop(['Runs_scored','Match','dismissed','Boundary','Balls_bowled'],axis=1,inplace=True)

In [67]:
final_data.replace(np.inf, 0, inplace=True)
final_data.replace(np.NaN,0,inplace=True)

In [68]:
final_data

,bowler,over_type,number_of_dotballs,Average,Boundary_per_wicket,Striker_rate,Balls_per_boundary
0,A Ashish Reddy,Death overs,18.0,15.375000,1.625000,9.375000,5.769231
1,A Ashish Reddy,Middle overs,59.0,25.700000,2.900000,18.900000,6.517241
2,A Ashish Reddy,PowerPlay,2.0,0.000000,0.000000,0.000000,1.500000
3,A Chandila,Death overs,1.0,0.000000,0.000000,0.000000,0.000000
4,A Chandila,Middle overs,32.0,100.000000,10.000000,84.000000,8.400000
...,...,...,...,...,...,...,...
1104,Yuvraj Singh,Death overs,19.0,16.166667,1.666667,11.166667,6.700000
1105,Yuvraj Singh,Middle overs,239.0,33.133333,3.000000,27.166667,9.055556
1106,Z Khan,Death overs,136.0,23.142857,2.571429,15.628571,6.077778
1107,Z Khan,Middle overs,100.0,37.200000,4.533333,25.533333,5.632353


In [72]:
data=pd.pivot_table(final_data,index=['bowler'],columns=['over_type']).reset_index()

In [73]:
data.head()

bowler     Average                          \
over_type                 Death overs Middle overs  PowerPlay   
0          A Ashish Reddy      15.375         25.7   0.000000   
1              A Chandila       0.000        100.0  13.900000   
2             A Choudhary      40.000         58.0  15.333333   
3             A Dananjaya         NaN          0.0   0.000000   
4              A Flintoff      52.000         33.0   0.000000   

          Balls_per_boundary                        Boundary_per_wicket  \
over_type        Death overs Middle overs PowerPlay         Death overs   
0                   5.769231     6.517241  1.500000               1.625   
1                   0.000000     8.400000  8.470588               0.000   
2                   6.750000     5.500000  5.285714               4.000   
3                        NaN     4.750000  3.000000                 NaN   
4                   2.250000    12.000000  4.000000               8.000   

                                 Striker_rate                          \
over_type Middle overs PowerPlay  Death overs Middle overs  PowerPlay   
0                  2.9  0.000000        9.375         18.9   0.000000   
1                 10.0  1.700000        0.000         84.0  14.400000   
2                  8.0  2.333333       27.000         44.0  12.333333   
3                  0.0  0.000000          NaN          0.0   0.000000   
4                  3.0  0.000000       18.000         36.0   0.000000   

          number_of_dotballs                         
over_type        Death overs Middle overs PowerPlay  
0                       18.0         59.0       2.0  
1                        1.0         32.0      70.0  
2                        7.0         17.0      18.0  
3                        NaN          4.0       2.0  
4                        1.0         16.0       3.0

In [74]:
data.replace(np.NaN,0,inplace=True)

In [75]:
data.head()

bowler     Average                          \
over_type                 Death overs Middle overs  PowerPlay   
0          A Ashish Reddy      15.375         25.7   0.000000   
1              A Chandila       0.000        100.0  13.900000   
2             A Choudhary      40.000         58.0  15.333333   
3             A Dananjaya       0.000          0.0   0.000000   
4              A Flintoff      52.000         33.0   0.000000   

          Balls_per_boundary                        Boundary_per_wicket  \
over_type        Death overs Middle overs PowerPlay         Death overs   
0                   5.769231     6.517241  1.500000               1.625   
1                   0.000000     8.400000  8.470588               0.000   
2                   6.750000     5.500000  5.285714               4.000   
3                   0.000000     4.750000  3.000000               0.000   
4                   2.250000    12.000000  4.000000               8.000   

                                 Striker_rate                          \
over_type Middle overs PowerPlay  Death overs Middle overs  PowerPlay   
0                  2.9  0.000000        9.375         18.9   0.000000   
1                 10.0  1.700000        0.000         84.0  14.400000   
2                  8.0  2.333333       27.000         44.0  12.333333   
3                  0.0  0.000000        0.000          0.0   0.000000   
4                  3.0  0.000000       18.000         36.0   0.000000   

          number_of_dotballs                         
over_type        Death overs Middle overs PowerPlay  
0                       18.0         59.0       2.0  
1                        1.0         32.0      70.0  
2                        7.0         17.0      18.0  
3                        0.0          4.0       2.0  
4                        1.0         16.0       3.0

In [76]:
data.columns = [' '.join(col).strip() for col in data.columns.values]

In [77]:
data.head()

,bowler,Average Death overs,Average Middle overs,Average PowerPlay,Balls_per_boundary Death overs,Balls_per_boundary Middle overs,Balls_per_boundary PowerPlay,Boundary_per_wicket Death overs,Boundary_per_wicket Middle overs,Boundary_per_wicket PowerPlay,Striker_rate Death overs,Striker_rate Middle overs,Striker_rate PowerPlay,number_of_dotballs Death overs,number_of_dotballs Middle overs,number_of_dotballs PowerPlay
0,A Ashish Reddy,15.375,25.7,0.000000,5.769231,6.517241,1.500000,1.625,2.9,0.000000,9.375,18.9,0.000000,18.0,59.0,2.0
1,A Chandila,0.000,100.0,13.900000,0.000000,8.400000,8.470588,0.000,10.0,1.700000,0.000,84.0,14.400000,1.0,32.0,70.0
2,A Choudhary,40.000,58.0,15.333333,6.750000,5.500000,5.285714,4.000,8.0,2.333333,27.000,44.0,12.333333,7.0,17.0,18.0
3,A Dananjaya,0.000,0.0,0.000000,0.000000,4.750000,3.000000,0.000,0.0,0.000000,0.000,0.0,0.000000,0.0,4.0,2.0
4,A Flintoff,52.000,33.0,0.000000,2.250000,12.000000,4.000000,8.000,3.0,0.000000,18.000,36.0,0.000000,1.0,16.0,3.0


In [78]:
data.replace(np.NaN, 0, inplace=True)

In [79]:
data.head()

,bowler,Average Death overs,Average Middle overs,Average PowerPlay,Balls_per_boundary Death overs,Balls_per_boundary Middle overs,Balls_per_boundary PowerPlay,Boundary_per_wicket Death overs,Boundary_per_wicket Middle overs,Boundary_per_wicket PowerPlay,Striker_rate Death overs,Striker_rate Middle overs,Striker_rate PowerPlay,number_of_dotballs Death overs,number_of_dotballs Middle overs,number_of_dotballs PowerPlay
0,A Ashish Reddy,15.375,25.7,0.000000,5.769231,6.517241,1.500000,1.625,2.9,0.000000,9.375,18.9,0.000000,18.0,59.0,2.0
1,A Chandila,0.000,100.0,13.900000,0.000000,8.400000,8.470588,0.000,10.0,1.700000,0.000,84.0,14.400000,1.0,32.0,70.0
2,A Choudhary,40.000,58.0,15.333333,6.750000,5.500000,5.285714,4.000,8.0,2.333333,27.000,44.0,12.333333,7.0,17.0,18.0
3,A Dananjaya,0.000,0.0,0.000000,0.000000,4.750000,3.000000,0.000,0.0,0.000000,0.000,0.0,0.000000,0.0,4.0,2.0
4,A Flintoff,52.000,33.0,0.000000,2.250000,12.000000,4.000000,8.000,3.0,0.000000,18.000,36.0,0.000000,1.0,16.0,3.0


In [81]:
max_avg_deathovers=max(data['Average Death overs'])
min_avg_deathovers=min(data['Average Death overs'])

max_avg_middleovers=max(data['Average Middle overs'])
min_avg_middleovers=min(data['Average Middle overs'])

max_avg_powerplay=max(data['Average PowerPlay'])
min_avg_powerplay=min(data['Average PowerPlay'])

max_bpb_deathovers=max(data['Balls_per_boundary Death overs'])
min_bpb_deathovers=min(data['Balls_per_boundary Death overs'])

max_bpb_middleovers=max(data['Balls_per_boundary Middle overs'])
min_bpb_middleovers=min(data['Balls_per_boundary Middle overs'])

max_bpb_powerplay=max(data['Balls_per_boundary PowerPlay'])
min_bpb_powerplay=min(data['Balls_per_boundary PowerPlay'])

max_bpw_deathovers=max(data['Boundary_per_wicket Death overs'])
min_bpw_deathovers=min(data['Boundary_per_wicket Death overs'])

max_bpw_middleovers=max(data['Boundary_per_wicket Middle overs'])
min_bpw_middleovers=min(data['Boundary_per_wicket Middle overs'])

max_bpw_powerplay=max(data['Boundary_per_wicket PowerPlay'])
min_bpw_powerplay=min(data['Boundary_per_wicket PowerPlay'])

max_sr_deathovers=max(data['Striker_rate Death overs'])
min_sr_deathovers=min(data['Striker_rate Death overs'])

max_sr_middleovers=max(data['Striker_rate PowerPlay'])
min_sr_middleovers=min(data['Striker_rate PowerPlay'])

max_sr_powerplay=max(data['Striker_rate PowerPlay'])
min_sr_powerplay=min(data['Striker_rate PowerPlay'])
                             

max_nod_deathovers=max(data['number_of_dotballs Death overs'])
min_nod_deathovers=min(data['number_of_dotballs Death overs'])

max_nod_middleovers=max(data['number_of_dotballs Middle overs'])
min_nod_middleovers=min(data['number_of_dotballs Middle overs'])

max_nod_powerplay=max(data['number_of_dotballs PowerPlay'])
min_nod_powerplay=min(data['number_of_dotballs PowerPlay'])


In [82]:
data.round(2)

,bowler,Average Death overs,Average Middle overs,Average PowerPlay,Balls_per_boundary Death overs,Balls_per_boundary Middle overs,Balls_per_boundary PowerPlay,Boundary_per_wicket Death overs,Boundary_per_wicket Middle overs,Boundary_per_wicket PowerPlay,Striker_rate Death overs,Striker_rate Middle overs,Striker_rate PowerPlay,number_of_dotballs Death overs,number_of_dotballs Middle overs,number_of_dotballs PowerPlay
0,A Ashish Reddy,15.38,25.70,0.00,5.77,6.52,1.50,1.62,2.90,0.00,9.38,18.90,0.00,18.0,59.0,2.0
1,A Chandila,0.00,100.00,13.90,0.00,8.40,8.47,0.00,10.00,1.70,0.00,84.00,14.40,1.0,32.0,70.0
2,A Choudhary,40.00,58.00,15.33,6.75,5.50,5.29,4.00,8.00,2.33,27.00,44.00,12.33,7.0,17.0,18.0
3,A Dananjaya,0.00,0.00,0.00,0.00,4.75,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,4.0,2.0
4,A Flintoff,52.00,33.00,0.00,2.25,12.00,4.00,8.00,3.00,0.00,18.00,36.00,0.00,1.0,16.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,YA Abdulla,16.50,54.50,14.71,4.83,4.69,5.24,2.00,8.00,2.43,9.67,37.50,12.71,14.0,26.0,48.0
425,YK Pathan,20.33,39.20,28.71,4.29,8.19,6.94,2.33,3.84,3.79,10.00,31.44,26.29,8.0,220.0,165.0
426,YS Chahal,24.27,22.53,31.31,4.26,8.62,5.04,3.09,2.14,4.62,13.18,18.47,23.31,33.0,612.0,158.0
427,Yuvraj Singh,16.17,33.13,0.00,6.70,9.06,0.00,1.67,3.00,0.00,11.17,27.17,0.00,19.0,239.0,0.0


In [83]:
data['avg_deathovers']=data['Average Death overs'].apply(lambda x:(x-min_avg_deathovers)/(max_avg_deathovers-min_avg_deathovers))
data['avg_middleovers']=data['Average Middle overs'].apply(lambda x:(x-min_avg_middleovers)/(max_avg_middleovers-min_avg_middleovers))
data['avg_powerplay']=data['Average PowerPlay'].apply(lambda x:(x-min_avg_powerplay)/(max_avg_powerplay-min_avg_powerplay))

data['bpb_deathovers']=data['Balls_per_boundary Death overs'].apply(lambda x:(x-min_bpb_deathovers)/(max_bpb_deathovers-min_bpb_deathovers))
data['bpb_middleovers']=data['Balls_per_boundary Middle overs'].apply(lambda x:(x-min_bpb_middleovers)/(max_bpb_middleovers-min_bpb_middleovers))
data['bpb_powerplay']=data['Balls_per_boundary PowerPlay'].apply(lambda x:(x-min_bpb_powerplay)/(max_bpb_powerplay-min_bpb_powerplay))

data['bpw_deathovers']=data['Boundary_per_wicket Death overs'].apply(lambda x:(x-min_bpw_deathovers)/(max_bpw_deathovers-min_bpw_deathovers))
data['bpw_middleovers']=data['Boundary_per_wicket Middle overs'].apply(lambda x:(x-min_bpw_middleovers)/(max_bpw_middleovers-min_bpw_middleovers))
data['bpw_powerplay']=data['Boundary_per_wicket PowerPlay'].apply(lambda x:(x-min_bpw_powerplay)/(max_bpw_powerplay-min_bpw_powerplay))

data['sr_deathovers']=data['Striker_rate Death overs'].apply(lambda x:(x-min_sr_deathovers)/(max_sr_deathovers-min_sr_deathovers))
data['sr_middleovers']=data['Striker_rate Middle overs'].apply(lambda x:(x-min_sr_middleovers)/(max_sr_middleovers-min_sr_middleovers))
data['sr_powerplay']=data['Striker_rate PowerPlay'].apply(lambda x:(x-min_sr_powerplay)/(max_sr_powerplay-min_sr_powerplay))


data['nod_deathovers']=data['number_of_dotballs Death overs'].apply(lambda x:(x-min_nod_deathovers)/(max_nod_deathovers-min_nod_deathovers))
data['nod_middleovers']=data['number_of_dotballs Middle overs'].apply(lambda x:(x-min_nod_middleovers)/(max_nod_middleovers-min_nod_middleovers))
data['nod_powerplay']=data['number_of_dotballs PowerPlay'].apply(lambda x:(x-min_nod_powerplay)/(max_nod_powerplay-min_nod_powerplay))


In [87]:
data['bowler'].unique()

array(['A Ashish Reddy', 'A Chandila', 'A Choudhary', 'A Dananjaya',
       'A Flintoff', 'A Kumble', 'A Mishra', 'A Mithun', 'A Nehra',
       'A Nel', 'A Nortje', 'A Singh', 'A Symonds', 'A Uniyal', 'A Zampa',
       'AA Chavan', 'AA Jhunjhunwala', 'AA Kazi', 'AA Noffke',
       'AB Agarkar', 'AB Dinda', 'AB McDonald', 'AC Gilchrist',
       'AC Thomas', 'AC Voges', 'AD Mascarenhas', 'AD Mathews',
       'AD Russell', 'AF Milne', 'AG Murtaza', 'AJ Finch', 'AJ Tye',
       'AL Menaria', 'AM Nayar', 'AM Rahane', 'AM Salvi', 'AN Ahmed',
       'AP Dole', 'AR Patel', 'AS Joseph', 'AS Rajpoot', 'AS Raut',
       'AS Roy', 'AUK Pathan', 'Abdul Samad', 'Abdur Razzak',
       'Abhishek Sharma', 'Anand Rajan', 'Ankit Sharma', 'Ankit Soni',
       'Anureet Singh', 'Arshdeep Singh', 'Avesh Khan', 'Azhar Mahmood',
       'B Akhil', 'B Chipli', 'B Geeves', 'B Kumar', 'B Laughlin',
       'B Lee', 'B Stanlake', 'BA Bhatt', 'BA Stokes', 'BAW Mendis',
       'BB Sran', 'BCJ Cutting', 'BE Hendricks',

In [89]:
bowler_name=input('Enter the name: ')
player_avg_deathovers=data[data.bowler==bowler_name].avg_deathovers.item()
player_avg_middleovers=data[data.bowler==bowler_name].avg_middleovers.item()
player_avg_powerplay=data[data.bowler==bowler_name].avg_powerplay.item()

player_bpb_deathovers=data[data.bowler==bowler_name].bpb_deathovers.item()
player_bpb_middleovers=data[data.bowler==bowler_name].bpb_middleovers.item()
player_bpb_powerplay=data[data.bowler==bowler_name].bpb_powerplay.item()

player_bpw_deathovers=data[data.bowler==bowler_name].bpw_deathovers.item()
player_bpw_middleovers=data[data.bowler==bowler_name].bpw_middleovers.item()
player_bpw_powerplay=data[data.bowler==bowler_name].bpw_powerplay.item()

player_sr_deathovers=data[data.bowler==bowler_name].sr_deathovers.item()
player_sr_middleovers=data[data.bowler==bowler_name].sr_middleovers.item()
player_sr_powerplay=data[data.bowler==bowler_name].sr_powerplay.item()

player_nod_deathovers=data[data.bowler==bowler_name].nod_deathovers.item()
player_nod_middleovers=data[data.bowler==bowler_name].nod_middleovers.item()
player_nod_powerplay=data[data.bowler==bowler_name].nod_powerplay.item()



avg_death1=np.array(data.avg_deathovers)
avg_death2= np.empty(len(avg_death1))
avg_death2.fill(player_avg_deathovers)

avg_middle1=np.array(data.avg_middleovers)
avg_middle2= np.empty(len(avg_middle1))
avg_middle2.fill(player_avg_middleovers)

avg_power1=np.array(data.avg_powerplay)
avg_power2= np.empty(len(avg_power1))
avg_power2.fill(player_avg_powerplay)

bpb_death1=np.array(data.bpb_deathovers)
bpb_death2= np.empty(len(bpb_death1))
bpb_death2.fill(player_bpb_deathovers)

bpb_middle1=np.array(data.bpb_middleovers)
bpb_middle2= np.empty(len(bpb_middle1))
bpb_middle2.fill(player_bpb_middleovers)

bpb_power1=np.array(data.bpb_powerplay)
bpb_power2= np.empty(len(bpb_power1))
bpb_power2.fill(player_bpb_powerplay)


bpw_death1=np.array(data.bpw_deathovers)
bpw_death2= np.empty(len(bpw_death1))
bpw_death2.fill(player_bpw_deathovers)

bpw_middle1=np.array(data.bpw_middleovers)
bpw_middle2= np.empty(len(bpw_middle1))
bpw_middle2.fill(player_bpw_middleovers)

bpw_power1=np.array(data.bpw_powerplay)
bpw_power2= np.empty(len(bpw_power1))
bpw_power2.fill(player_bpw_powerplay)

sr_death1=np.array(data.sr_deathovers)
sr_death2= np.empty(len(sr_death1))
sr_death2.fill(player_sr_deathovers)

sr_middle1=np.array(data.sr_middleovers)
sr_middle2= np.empty(len(sr_middle1))
sr_middle2.fill(player_sr_middleovers)

sr_power1=np.array(data.sr_powerplay)
sr_power2= np.empty(len(sr_power1))
sr_power2.fill(player_sr_powerplay)

nod_death1=np.array(data.nod_deathovers)
nod_death2= np.empty(len(nod_death1))
nod_death2.fill(player_nod_deathovers)

nod_middle1=np.array(data.nod_middleovers)
nod_middle2= np.empty(len(nod_middle1))
nod_middle2.fill(player_nod_middleovers)

nod_power1=np.array(data.nod_powerplay)
nod_power2= np.empty(len(nod_power1))
nod_power2.fill(player_nod_powerplay)



avg_death3=abs(avg_death1-avg_death2)**2
avg_middle3=abs(avg_middle1-avg_middle2)**2
avg_power3=abs(avg_power1-avg_power2)**2

bpb_death3=abs(bpb_death1-bpb_death2)**2
bpb_middle3=abs(bpb_middle1-bpb_middle2)**2
bpb_power3=abs(bpb_power1-bpb_power2)**2

bpw_death3=abs(bpw_death1-bpw_death2)**2
bpw_middle3=abs(bpw_middle1-bpw_middle2)**2
bpw_power3=abs(bpw_power1-bpw_power2)**2

sr_death3=abs(sr_death1-sr_death2)**2
sr_middle3=abs(sr_middle1-sr_middle2)**2
sr_power3=abs(sr_power1-sr_power2)**2

nod_death3=abs(nod_death1-nod_death2)**2
nod_middle3=abs(nod_middle1-nod_middle2)**2
nod_power3=abs(nod_power1-nod_power2)**2


distance=avg_death3+avg_middle3+avg_power3+bpb_death3+bpb_middle3+bpb_power3+bpw_death3+bpw_middle3+bpw_power3+sr_death3+sr_middle3+sr_power3+nod_death3+nod_middle3+nod_power3
dist=np.sqrt(distance)
data['distance']=dist
final_data=data.drop(['Average Death overs', 'Average Middle overs',
       'Average PowerPlay', 'Balls_per_boundary Death overs',
       'Balls_per_boundary Middle overs', 'Balls_per_boundary PowerPlay',
       'Boundary_per_wicket Death overs', 'Boundary_per_wicket Middle overs',
       'Boundary_per_wicket PowerPlay', 'Striker_rate Death overs',
       'Striker_rate Middle overs', 'Striker_rate PowerPlay',
       'number_of_dotballs Death overs', 'number_of_dotballs Middle overs',
       'number_of_dotballs PowerPlay'],axis=1)
min_d = min(final_data.distance)
max_d = max(final_data.distance)

final_data['similarity'] = final_data['distance'].apply(lambda x: 100*(1 - (x-min_d)/(max_d-min_d)))
final_data.sort_values('similarity', ascending= False).head(10)


Enter the name: JC Archer


,bowler,avg_deathovers,avg_middleovers,avg_powerplay,bpb_deathovers,bpb_middleovers,bpb_powerplay,bpw_deathovers,bpw_middleovers,bpw_powerplay,sr_deathovers,sr_middleovers,sr_powerplay,nod_deathovers,nod_middleovers,nod_powerplay,distance,similarity
154,JC Archer,0.200257,0.090476,0.134948,0.372840,0.186111,0.424641,0.157895,0.079412,0.107843,0.264912,0.248148,0.257807,0.280702,0.083333,0.255370,0.000000,100.000000
267,NM Coulter-Nile,0.170732,0.077381,0.234163,0.335135,0.158784,0.296575,0.146825,0.077731,0.201923,0.221429,0.207231,0.337132,0.178947,0.098394,0.207637,0.247284,90.043319
239,MJ McClenaghan,0.156973,0.133220,0.205645,0.351913,0.137097,0.276923,0.130342,0.130252,0.174731,0.206410,0.299824,0.272401,0.319298,0.110442,0.365155,0.248517,89.993670
190,KK Cooper,0.189024,0.092186,0.211397,0.461261,0.196774,0.375940,0.128472,0.070136,0.145833,0.266667,0.231719,0.308642,0.238596,0.073293,0.051313,0.255035,89.731237
281,PJ Cummins,0.216556,0.122024,0.256863,0.335135,0.182895,0.279778,0.186869,0.093137,0.211111,0.281818,0.286008,0.332510,0.189474,0.081325,0.210024,0.260607,89.506866
324,S Aravind,0.121326,0.105820,0.199632,0.359420,0.126136,0.306094,0.098291,0.107843,0.158333,0.158974,0.228395,0.272840,0.136842,0.064257,0.223150,0.263300,89.398441
256,Mohammed Siraj,0.209124,0.138889,0.208180,0.331034,0.166304,0.229730,0.179012,0.122995,0.192708,0.266667,0.343434,0.249228,0.266667,0.113454,0.183771,0.269908,89.132393
308,RJ Harris,0.132791,0.120748,0.252101,0.356944,0.152885,0.328216,0.111111,0.109244,0.214286,0.178472,0.280423,0.395944,0.256140,0.055221,0.281623,0.271908,89.051849
31,AJ Tye,0.147625,0.128571,0.149064,0.354472,0.197656,0.277512,0.119883,0.094118,0.125000,0.191228,0.312346,0.195286,0.178947,0.059237,0.091885,0.282161,88.639001
244,MM Patel,0.203704,0.073192,0.198783,0.263542,0.208636,0.289042,0.197531,0.059913,0.175287,0.234259,0.209877,0.285228,0.238596,0.174699,0.415274,0.283852,88.570920
